In [2]:
# defining variables and reference files needed 
TRAIN_DATA = '/Users/wesley/Desktop/4579_final/MRPC/train_data.csv'
TEST_DATA = '/Users/wesley/Desktop/4579_final/MRPC/test_data.csv'

VOCAB_FILE = './vocab_mrpc.txt'
MODEL_FILE = './dssm_pr.pkl'

N = 3
SENTENCE_MAXLEN = 90
CHAR_SIZE = 10041
EMBEDDING_SIZE = 300

EPOCH = 100
BATCH_SIZE = 50
LR = 0.0005

In [2]:
!pip uninstall torch

Found existing installation: torch 2.0.0
Uninstalling torch-2.0.0:
  Would remove:
    /Users/wesley/opt/anaconda3/bin/convert-caffe2-to-onnx
    /Users/wesley/opt/anaconda3/bin/convert-onnx-to-caffe2
    /Users/wesley/opt/anaconda3/bin/torchrun
    /Users/wesley/opt/anaconda3/lib/python3.9/site-packages/functorch/*
    /Users/wesley/opt/anaconda3/lib/python3.9/site-packages/torch-2.0.0.dist-info/*
    /Users/wesley/opt/anaconda3/lib/python3.9/site-packages/torch/*
    /Users/wesley/opt/anaconda3/lib/python3.9/site-packages/torchgen/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [3]:
import numpy as np
import pandas as pd
import torch


In [ ]:
## create N_gram vector given a word
def n_gram(word,n = N):
    s = []
    word = '#' + word + '#'
    for i in range(len(word)-2):
        s.append(word[i:i+n]) ## letter 3_gram 
    return s

In [4]:
## create list of words in a sentence
def lst_gram(lst,n = N):
    s = []
    for word in str(lst).lower().split():
        s.extend(n_gram(word, n))
    return s

In [5]:
vocab = []  # initiate a vector to store letter n_grams
file_path = './MRPC/'  # data set path
files = ['train_data.csv','test_data.csv']  # training data & testing data set

In [6]:
# process train and test files
for file in files:
    f = open(file_path + file,encoding = 'utf-8').readlines()
    for i in range(1,len(f)):  # line 0 is header
        s1,s2 = f[i][2:].strip('\n').split('\t')
        # delete the first 3 character of each line, tab space, and line change space, parse into two sentences
        vocab.extend(lst_gram(s1))
        vocab.extend(lst_gram(s2))

FileNotFoundError: [Errno 2] No such file or directory: './MRPC/train_data.csv'

In [7]:
vocab = set(vocab)
vocab_list = ['[PAD]', '[UNK]']
vocab_list.extend(list(vocab))

In [8]:
with open(VOCAB_FILE, 'w', encoding = 'utf-8') as f:
    for slice in vocab_list:
        f.write(slice)
        f.write('\n')

In [9]:
def load_vocab():
    vocab = open(VOCAB_FILE,encoding='utf-8').readlines()
    slice2idx = {}
    idx2slice = {}
    count = 0
    for char in vocab:
        char = char.strip('\n')
        slice2idx[char] = cnt
        idx2slice[count] = char
        count += 1
    return slice2idx, idx2slice

In [10]:
def padding(text, maxlen = SENTENCE_MAXLEN):
    pad_text = []
    for sentence in text:
        pad_sentence = np.zeros(maxlen).astype('int64') #build a all zero vector that matches the shape of text
        count = 0
        for index in sentence:
            pad_sentence[count] = index
            count += 1
            if count == maxlen:
                break
        pad_text.append(pad_sentence.tolist())
    return pad_text


In [11]:
def char_index(text_a,text_b):
    slice2idx, idx2slice = load_vocab()
    a_list,b_list = [],[]
    
    for a_sentence,b_sentence in zip(text_a,text_b): # for each line (parsed into two sentences)in the file
        a,b = [],[]
        for slice in lst_gram(a_sentence): # for each slice of the first sentence in each line
            if slice in slice2idx.keys():
                a.append(slice2idx[slice])
            else:
                a.append(1)
                
        for slice in lst_gram(b_sentence): # for each slice of the second sentence in each line
            if slice in slice2idx.keys():
                b.append(slice2idx[slice])
            else:
                b.append(1)
                
        a_list.append(a)
        b_list.append(b)
        
    a_list = padding(a_list)
    b_list = padding(b_list)
        
    return a_list,b_list


In [12]:
def load_char_data(filename):
    df = pd.read_csv(filename, sep = '\t')
    text_a = df['#1 string'].values
    text_b = df['#2 string'].values
    label = df['quality'].values
    a_index,b_index = char_index(text_a,text_b)
    return np.array(a_index),np.array(b_index),np.array(label)